In [4]:
import ipywidgets as widgets
from traitlets import Unicode

*DOMWIDGET* subclass is defined in python, the instance lives on the server, is related to a *DOMWidgetView* javascript object that lives in the browser.
  has three traitlet slots which tie together the js and py objects.
Here, we specify the browser's js module name and the name of the js widget's view.
value is the (arbitrarily named?) python object state slot.
tag these three fields so that they are synced between server & browser.

(`value` was the origin name of the third slot, changed here (and below) to `valor` to test out the name-flexibility of syncing data between kernel and browser.)

In [13]:
class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    valor = Unicode('Hello World, with valor!').tag(sync=True)

**requirejs**: a popular library for managing (loading, defining) modules.

`define('moduleName", [other libraries], function(refForLibrary1, refForLibrary2, ...){})`

The only exported items in the hello module defined below is a js subclass of ipywidgets  DOMWidgetView, with specialized methods `render` and `value_changed`.

`DOMWidgetView` provides a `model` slot, with a get method (see `this.model.get(slotName)` below) and the ability to assign at least an on-change event.

I don't know how and when these methods are called.

In [17]:
%%javascript
require.undef('hello');

define('hello', ["jupyter-js-widgets"], function(widgets) {
  var HelloView = widgets.DOMWidgetView.extend({
     render: function() { 
       this.value_changed();
       this.model.on('change:valor', this.value_changed, this);
       },
     value_changed: function() {
       this.el.textContent = this.model.get('valor'); 
       },
    });
   return {
     HelloView: HelloView
     };
  });

<IPython.core.display.Javascript object>

In [18]:
w = HelloWidget()
w

In [24]:
w.valor = 'test, a new valor value'

In [26]:
w.valor

'test, a new valor value'

In [28]:
x = Unicode("abc")

In [29]:
from traitlets import CInt

class SpinnerWidget(widgets.DOMWidget):
    _view_name = Unicode('SpinnerView').tag(sync=True)
    _view_module = Unicode('spinner').tag(sync=True)
    value = CInt().tag(sync=True)

## An exploration of traitlets

An add-on module, apparently used mostly for ipywidgets, [traitlets](https://github.com/ipython/traitlets) provide stronlgy typed attributes upon which notifications can be hung.  The implementation relies on the [descriptor](https://docs.python.org/3/howto/descriptor.html) pattern. This package powers the configuration system of IPython and Jupyter, and the declarative API of IPython interactive widgets

In [32]:
x.default_value

'abc'

In [ ]:
%%javascript
define('spinner', ["jupyter-js-widgets"], function(widgets) {
    
    var SpinnerView = widgets.DOMWidgetView.extend({
        
        render: function() { 
            
            // jQuery code to create a spinner and append it to $el
            this.input = $('<input />');
            this.el.append(this.input);
            this.spinner = this.input.spinner({
                change: function( event, ui ) {}
            });
            
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },
        
        value_changed: function() {
            
        },
    });
    
    return {
        SpinnerView: SpinnerView
    };
});

In [ ]:
%%javascript
requirejs.undef('spinner');

define('spinner', ["jupyter-js-widgets"], function(widgets) {

    var SpinnerView = widgets.DOMWidgetView.extend({
        render: function() { 

            var that = this;
            this.$input = $('<input />');
            this.$el.append(this.$input);
            this.$spinner = this.$input.spinner({
                change: function( event, ui ) {
                    that.handle_spin(that.$spinner.spinner('value'));
                },
                spin: function( event, ui ) {
                    //ui.value is the new value of the spinner
                    that.handle_spin(ui.value);
                }
            });
            
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },
        
        value_changed: function() {
            this.$spinner.spinner('value', this.model.get('value'));
        },
        
        handle_spin: function(value) {
            this.model.set('value', value);
            this.touch();
        },
    });
    
    return {
        SpinnerView: SpinnerView
    };
});

In [ ]:
w = SpinnerWidget(value=5)
w

In [ ]:
w.value

In [ ]:
w.value = 20

In [ ]:
from IPython.display import display
w1 = SpinnerWidget(value=0)
w2 = widgets.IntSlider()
display(w1,w2)

from traitlets import link
mylink = link((w1, 'value'), (w2, 'value'))